In [8]:
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import re

df_column = ["type (actionDetails 0)",
    "url (actionDetails 0)",
    "pageTitle (actionDetails 0)", 
    "pageIdAction (actionDetails 0)" 
        
    ]

def copy_from_stringio(conn, df, person):
    buffer = StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)

    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    

# URL to fetch the data
url = "https://demo.matomo.cloud/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=yesterday&format=Tsv&token_auth=anonymous&translateColumnNames=1"
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    pattern = re.compile(r'\(actionDetails \d+\)')
    filtered_columns = [column for column in df.columns if pattern.search(column)]

   
    if len(filtered_columns) > 1600:
        filtered_columns = filtered_columns[:1600]

    
    df_filtered = df[filtered_columns]
    
  

    # Optional: Print the list of found columns
    # print(f"Filtered columns: {filtered_columns}")

    # Database connection settings
    username = 'postgres'
    password = 'Sharjeel'
    host = 'localhost'
    port = '5432'
    database = 'Pharma'

    # Create the database connection engine
    engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')
    
    # Specify the table name
    table_name = 'person'
    
    # Upload the DataFrame to PostgreSQL
    df_filtered.to_sql(table_name, engine, if_exists='replace', index=False)

    print(f"Data successfully uploaded to the '{table_name}' table in PostgreSQL.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Data successfully uploaded to the 'person' table in PostgreSQL.


In [9]:
import requests
import pandas as pd
from io import StringIO
from sqlalchemy import create_engine
import re
url = "https://demo.matomo.cloud/?module=API&method=Live.getLastVisitsDetails&idSite=1&period=day&date=yesterday&format=Tsv&token_auth=anonymous&translateColumnNames=1"
response = requests.get(url)
response

<Response [200]>